In [1]:
from train_setup import *

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


loading data from ../aa/tumors.pkl
loading data from ../data_processing/consolidated_data.pkl
loading data from ../data_processing/dna_seq_by_hgncId.pkl
loading data from ../aa/canonical_ref.pkl
loading data from ../aa/canonical_mut.pkl
loading data from ../aa/assays.pkl


In [ ]:
# snv150_df = get_snv150_df(class_data, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_df.csv')
# snv150_msk468_df = get_snv150_df(class_data_msk468, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_msk468_df.csv')
# snv150_df_withPos = get_snv150_withPos_df(class_data, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_df_withPos.csv')
# snv150_msk468_df_withPos = get_snv150_withPos_df(class_data_msk468, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_msk468_df_withPos.csv')

# snv150_survival_df = get_snv150_df(survival_data, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_survival_df.csv')
# snv150_msk468_survival_df = get_snv150_df(survival_data_msk468, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_msk468_survival_df.csv')
# snv150_survival_df_withPos = get_snv150_withPos_df(survival_data, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_survival_df_withPos.csv')
# snv150_msk468_survival_df_withPos = get_snv150_withPos_df(survival_data_msk468, data, tumors, seqs, ref_aa, mut_aa).to_csv('snv150_msk468_survival_df_withPos.csv')

snv150_survival_df = pd.read_csv('snv150_survival_df.csv', index_col=0)
snv150_msk468_survival_df = pd.read_csv('snv150_msk468_survival_df.csv', index_col=0)
snv150_survival_df_withPos = pd.read_csv('snv150_survival_df_withPos.csv', index_col=0)
snv150_msk468_survival_df_withPos = pd.read_csv('snv150_msk468_survival_df_withPos.csv', index_col=0)

snv150_df = pd.read_csv('snv150_df.csv', index_col=0)
snv150_msk468_df = pd.read_csv('snv150_msk468_df.csv', index_col=0)
snv150_df_withPos = pd.read_csv('snv150_df_withPos.csv', index_col=0)
snv150_msk468_df_withPos = pd.read_csv('snv150_msk468_df_withPos.csv', index_col=0)

In [ ]:
# snv150_dfs_with_somatt = get_snv150_df(class_data, data, tumors, seqs, ref_aa, mut_aa, return_somatt_df=True)
# snv150_msk468_dfs_with_somatt = get_snv150_df(class_data_msk468, data, tumors, seqs, ref_aa, mut_aa, return_somatt_df=True)
# snv150_dfs_withPos_with_somatt = get_snv150_withPos_df(class_data, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)
# snv150_msk468_dfs_withPos_with_somatt = get_snv150_withPos_df(class_data_msk468, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)

# snv150_survival_dfs_with_somatt = get_snv150_df(survival_data, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)
# snv150_msk468_survival_dfs_with_somatt = get_snv150_df(survival_data_msk468, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)
# snv150_survival_dfs_withPos_with_somatt = get_snv150_withPos_df(survival_data, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)
# snv150_msk468_survival_dfs_withPos_with_somatt = get_snv150_withPos_df(survival_data_msk468, data, tumors, seqs, ref_aa, mut_aa,  return_somatt_df=True)

100%|██████████| 143530/143530 [00:07<00:00, 19633.70it/s]


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 1) Merge label (CANCER_TYPE) onto snv150_df
#    snv150_df is indexed by barcode, so we merge on the index vs. class_data['barcode']
def run_rf(snv150_df):
    df_merged = snv150_df.merge(
        class_data[['barcode', 'CANCER_TYPE']], 
        left_index=True, 
        right_on='barcode'
    )

    # 2) Separate features (the 150 SNV columns) and the label (CANCER_TYPE).
    X = df_merged[snv150_df.columns]  # the 150 mutation features
    y = df_merged['CANCER_TYPE']

    # 3) Split into train/test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42
    )

    # 4) Train a Random Forest
    rf = RandomForestClassifier(
        n_estimators=100, 
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_train, y_train)

    # 5) Predict and evaluate
    y_pred = rf.predict(X_test)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=np.nan))

run_rf(snv150_df)
run_rf(snv150_msk468_df)

Accuracy: 0.31791263150560856
                              precision    recall  f1-score   support

              Bladder Cancer       0.39      0.35      0.37       992
               Breast Cancer       0.26      0.34      0.29      3084
   Cancer of Unknown Primary       0.08      0.01      0.02      1110
           Colorectal Cancer       0.30      0.48      0.37      3441
          Endometrial Cancer       0.31      0.13      0.18      1199
      Esophagogastric Cancer       0.13      0.03      0.05       957
                      Glioma       0.26      0.25      0.25      2252
        Head and Neck Cancer       0.03      0.00      0.01       407
        Hepatobiliary Cancer       0.11      0.03      0.05       720
                    Leukemia       0.28      0.19      0.23       808
     Mature B-Cell Neoplasms       0.29      0.11      0.16      1137
                    Melanoma       0.53      0.57      0.55      1357
Myeloproliferative Neoplasms       0.49      0.59      0.54

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim

class DNN(nn.Module):
    def __init__(self, 
                 in_features, 
                 out_features,
                 emb_dim = 640, 
                 dropout_rate=1e-6, 
                 activation=nn.ReLU):
        super(DNN, self).__init__()
        
        self.act = activation()
        self.drop = nn.Dropout(p=dropout_rate)
        self.out_features = out_features
        
        self.fc1 = nn.Linear(in_features, in_features)
        self.fc2 = nn.Linear(in_features, emb_dim)
        self.fc3 = nn.Linear(emb_dim, emb_dim)
        self.norm = nn.LayerNorm(emb_dim)
        self.fc_out = nn.Linear(emb_dim, out_features)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        
        x = self.fc2(x)
        x = self.act(x)
        x = self.drop(x)
        
        x = self.fc3(x)
        x = self.act(x)
        x = self.drop(x)

        if self.out_features==1: x = self.norm(x)
        
        x = self.fc_out(x)
        return x

In [6]:
def run_experiment(
    data_splits, 
    model, 
    num_epochs=100, 
    learning_rate = .001,
    l2_penalty=.001, 
    device="cpu",
    survival = False
    # model_type="sklearn"
):

    if survival:
        X_train, X_test, time_train, time_test, event_train, event_test = data_splits
        criterion = negative_log_partial_likelihood
    else:
        X_train, X_test, y_train, y_test = data_splits
        criterion = nn.CrossEntropyLoss()
        X_train_tensor = torch.FloatTensor(X_train.values).to(device)
        y_train_tensor = torch.LongTensor(y_train.factorize()[0]).to(device)
        classes = y_train.factorize()[1]

    # if model_type == "sklearn":
    #     model.fit(X_train, y_train)
    #     y_pred = model.predict(X_test)

    # elif model_type == "DNN":
    model.to(device)
    
    optimizer = optim.Adam(
        model.parameters(), 
        lr=learning_rate, 
        weight_decay=l2_penalty
    )

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}")

    # 5b) Predict on X_test
    model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test.values).to(device)
        outputs_test = model(X_test_tensor)
        _, predicted_labels = torch.max(outputs_test, dim=1)

    # Convert predicted label indices back to the original class names
    y_pred = classes[predicted_labels.cpu()]

    # Common evaluation
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
run_fusion(
    class_data, 
    snv150_dfs_with_somatt,
    mut_embeddings,
    ref_embeddings,
    tumors,
    assays,
    survival=False
    )

143530 samples
129608 unique patients
20 labels

FOLD 1


TRAINING:   1%|          | 2/230 [00:06<11:42,  3.08s/it, Epoch=1/2, Loss: 2.9997]


KeyboardInterrupt: 

In [12]:
d = survival_data[survival_data['assay']=='MSK-IMPACT468']
all_genes_468 = list(get_gene_indexes(d).keys())
fusion_dim = 150 + len(all_genes_468)
print(f'fusion dim {fusion_dim}')

run_fusion(
    class_data_msk468, 
    snv150_msk468_dfs_withPos_with_somatt,
    mut_embeddings,
    ref_embeddings,
    tumors,
    assays,
    fusion_dim=fusion_dim,
    num_epochs = 10,
    survival=False
    )

fusion dim 594
29289 samples
26663 unique patients
18 labels

FOLD 1


TESTING: 100%|██████████| 12/12 [00:14<00:00,  1.23s/it]


Test Accuracy: 38.45%, (2253 of 5860)


TESTING: 100%|██████████| 12/12 [00:16<00:00,  1.34s/it]

Test Accuracy: 50.24%, (2944 of 5860)
Best Accuracy: 50.24% at epoch 1


In [ ]:
run_dnn(
    class_data = survival_data, 
    df = snv150_df,
    survival=True
    )

36722 unique patients

FOLD 1


TESTING: 100%|██████████| 16/16 [00:00<00:00, 60.56it/s]


C-Index: 0.5798


TESTING: 100%|██████████| 16/16 [00:00<00:00, 60.40it/s]


C-Index: 0.5908
Best C-Index: 0.5908 at epoch 1


In [23]:
run_dnn(
    class_data = survival_data[survival_data['assay']=='MSK-IMPACT468'],
    df = snv150_msk468_survival_df,
    survival=True
    )

5425 unique patients

FOLD 1


TRAINING:   0%|          | 0/9 [00:00<?, ?it/s]

TESTING: 100%|██████████| 3/3 [00:00<00:00, 78.56it/s]


C-Index: nan


TESTING: 100%|██████████| 3/3 [00:00<00:00, 78.51it/s]

C-Index: nan
Best C-Index: 0.0000 at epoch 0


In [24]:
all_genes = list(get_gene_indexes(survival_data).keys())
input_dim = 150 + len(all_genes)
print(f'input dim {input_dim}')

run_dnn(
    class_data = survival_data, 
    df = snv150_survival_df_withPos,
    input_dim = input_dim,
    survival=True,
    num_epochs = 2
    )

input dim 1439
41117 unique patients

FOLD 1


TESTING: 100%|██████████| 18/18 [00:00<00:00, 53.07it/s]


C-Index: 0.6856


TESTING: 100%|██████████| 18/18 [00:00<00:00, 53.00it/s]

C-Index: 0.6919
Best C-Index: 0.6919 at epoch 1


In [25]:
d = survival_data[survival_data['assay']=='MSK-IMPACT468']
all_genes_468 = list(get_gene_indexes(d).keys())
input_dim = 150 + len(all_genes_468)
print(f'input dim {input_dim}')

run_dnn(
    class_data = d, 
    df = snv150_msk468_survival_df_withPos,
    input_dim = input_dim,
    survival=True
    )

input dim 594
5425 unique patients

FOLD 1


TESTING: 100%|██████████| 3/3 [00:00<00:00, 74.43it/s]


C-Index: nan


TESTING: 100%|██████████| 3/3 [00:00<00:00, 74.70it/s]

C-Index: nan
Best C-Index: 0.0000 at epoch 0


In [2]:
# from train_setup import *

data_snv150 = lambda x: get_snv150_df(x, data, tumors, seqs, ref_aa, mut_aa, return_somatt_df=True)
data_snv150_pos = lambda x: get_snv150_withPos_df(x, data, tumors, seqs, ref_aa, mut_aa, return_somatt_df=True)

# snv
df_class = [
    data_snv150(class_data), 
    data_snv150_pos(class_data)
]

df_class_msk = [
    data_snv150(class_data_msk468),
    data_snv150_pos(class_data_msk468)
] 

df_surv = [
    data_snv150(survival_data),
    data_snv150_pos(survival_data)
]

df_surv_msk = [
    data_snv150(survival_data_msk468),
    data_snv150_pos(survival_data_msk468)
]

def get_fusion_dim(d,pos=False):
    if not pos: return 150
    all_genes = list(get_gene_indexes(d, tumors, data, seqs, ref_aa, mut_aa).keys())
    return 150 + len(all_genes)


def run(dfs, d, surv, e, names, lr = None):
    pos = False
    for c,n in zip(dfs,names):
        print(f'\n-------------------------------\n\n{n}\n')
        fusion_dim = get_fusion_dim(d, pos = pos)
        run_fusion(
            d, 
            c,
            mut_embeddings,
            ref_embeddings,
            tumors,
            assays,
            fusion_dim=fusion_dim,
            survival=surv,
            num_epochs=e,
            saveName = n,
            lr = lr
            )
        pos = not pos

# run(df_class, class_data, False, lr = .001, e = 15, names = ['snv150_class_fusion','snv150_pos_class_fusion'])
# run(df_class_msk, class_data_msk468, False, lr = .001, e = 15, names = ['snv150_class_fusion_msk468','snv150_pos_class_fusion_msk468'])   
# run(df_surv, survival_data, True, lr = .0001, e = 5, names = ['snv150_survival_fusion','snv150_pos_survival_fusion'])
run(df_surv_msk, survival_data_msk468, True, lr = .0001, e = 5, names = ['snv150_survival_fusion_msk468','snv150_pos_survival_fusion_msk468'])

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


loading data from ../aa/tumors.pkl
loading data from ../data_processing/consolidated_data.pkl
loading data from ../data_processing/dna_seq_by_hgncId.pkl
loading data from ../aa/canonical_ref.pkl
loading data from ../aa/canonical_mut.pkl
loading data from ../aa/assays.pkl


100%|██████████| 5547/5547 [00:00<00:00, 13308.05it/s]



-------------------------------

snv150_survival_fusion_msk468

5547 samples
5425 unique patients
53 cancer types
297 detailed cancer types


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.